In [1]:
%pylab inline
import warnings
from statsmodels.tools.sm_exceptions import HessianInversionWarning
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings('ignore')
import warnings
warnings.simplefilter('once', category=UserWarning)
warnings.simplefilter('once', category=HessianInversionWarning)
warnings.simplefilter('once', category=ConvergenceWarning)


                
                
from collections import defaultdict
from scipy.stats import nbinom
from scipy import optimize
from scipy.special import digamma
from scipy.special import polygamma
from pyfastnbfit import fastfit_negbin
from scvi.core.distributions import NegativeBinomial

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [31]:
def gphi(phi, y):
    #print(len(y))
    ybar = np.mean(y)
    n = len(y)
    sumterm = 0
    for i in range(n):
        for v in range(int(y[i])):
            sumterm += v/(1+phi*v)
    g2 = np.log(1+phi*ybar)
    g1 = ybar - phi/n*sumterm
    return g2/g1
    
def fit_nb_fixedpoint(y, max_iter=2000, xtol=1e-3):
    ybar = np.mean(y)
    p_y_gt_1 = len(y[y>=1]) /len(y)
    phi_init = 1/p_y_gt_1 - 1/ybar
    print(phi_init)
    phi = optimize.fixed_point(gphi, phi_init, args=([y]), xtol=xtol, maxiter=max_iter)
    return {"mu": ybar, "theta": 1/phi}

In [34]:
nb = NegativeBinomial(mu=20, theta=0.2)
y = nb.sample((200,)).numpy()
fit_nb_fixedpoint(y)

2.0613650029404993


RuntimeError: Failed to converge after 2000 iterations, value is nan

In [35]:
nb = NegativeBinomial(mu=20, theta=0.2)
y = nb.sample((200,)).numpy()
fit_nb_fixedpoint(y)

1.5998562308098776


RuntimeError: Failed to converge after 2000 iterations, value is nan

In [29]:
len(y[y>=1])

98

In [14]:
fit_nb_fixedpoint(y)

KeyboardInterrupt: 

In [12]:
fit_nb_fixedpoint(y)

KeyboardInterrupt: 

In [4]:
def trigamma(x):
    return polygamma(1,x)

def gamma_fit(x, s=None, limit=10):
    if s is None:
        m = np.mean(x)
        s = np.log(m) - np.mean(np.log(x))
    else:
        # sufficient stats
        m = x
    
    a = 0.5/s
    
    for i in range(limit):
        old_a = a
        g = log(a)-s-digamma(a)
        h = 1/a - trigamma(a)
        a = 1/(1/a + g/(a**2*h))
        if(np.abs(a - old_a) < 1e-8):
            break
    b = m/a
    return {"a":a, "b":b}


def fastnegbin_fit(y, w = None, limit=10):
    if w is None:
        w = [1]*len(y)
        
    w = w/np.sum(w)
    m = np.sum(w*y)
    v = np.sum(w*(y - m)**2);
    b = v/m - 1
    if b < 0:
        b <- 1e-3
    
    a = m/b
    for i in range(limit):
        old_a = a
        # E step
        p = b/(1+b)
        m = np.sum(w*(y+a)*p)
        s = np.log(m) - np.sum(w*(digamma(y+a) + np.log(p)))
        # M step
        gfit = gamma_fit(m, s)
        a = gfit["a"]       
        b = gfit["b"]
        
        if np.abs(a - old_a) < 1e-8:
            break  
    
    
    return {"mu":m, "theta":a}


In [5]:
import statsmodels.discrete.discrete_model as dm
from scipy.optimize import minimize

from scipy.special import digamma, polygamma, gammaln
from scipy.special import psi
from scipy.special import factorial
from scipy.optimize import fmin_l_bfgs_b as optim

In [6]:
def fit_nbinom_lbfgs(x, rmin=1e-1):
    """
    :param kin:
    :return:
    """
    k = np.array(x, dtype=np.float)
    N = len(k)
    sumx = np.sum(k)
    xbar = sumx / N

    def nll(r):
        return (
            -np.sum(gammaln(r + k))
            + N * gammaln(r)
            - N * r * np.log(r / (xbar + r))
            - sumx * np.log(xbar / (xbar + r))
        )

    r = minimize(nll, 1.0, bounds=[(rmin, None)], method="L-BFGS-B")
    r = r.x
    q = r / (r + np.sum(k / N))  # Note: this `p` = 1 - `p` from Wikipedia
    p = 1 - q
    return {"r": r, "p": p}


In [4]:
nb = NegativeBinomial(mu=10, theta=1/2)
y = nb.sample((200,)).numpy()
fastfit_negbin(y, max_iterations=100)

{'theta': 0.4701019654086743,
 'dispersion': 2.1271980837830973,
 'mu': 9.695000648498535,
 'iterations': 58}

In [110]:
nb = NegativeBinomial(mu=10, theta=1/2)
y = nb.sample((200,)).numpy()
fastnegbin_fit(y)

{'mu': 10.194999955282313, 'theta': 0.44336981103243495}

In [5]:
fastfit_negbin(y, max_iterations=230)

{'theta': 0.4701019654086743,
 'dispersion': 2.1271980837830973,
 'mu': 9.695000648498535,
 'iterations': 58}

In [6]:
fastfit_negbin(y, max_iterations=20)

{'theta': 5102.559627035662,
 'dispersion': 0.00019598007139427613,
 'mu': 9.695003509521484,
 'iterations': 20}

In [5]:
nb = NegativeBinomial(mu=100, theta=1/2)
y = nb.sample((200,)).numpy()
fastfit_negbin(y, max_iterations=100)

{'dispersion': 0.6207604722052953, 'mu': 108.23500061035156, 'iterations': 20}

In [26]:
nb = NegativeBinomial(mu=0.01, theta=1/2)
y = nb.sample((200,)).numpy()
fastfit_negbin(y, max_iterations=100)

{'dispersion': 0.9376011392175008,
 'mu': 0.014999994076788425,
 'iterations': 42}

In [20]:
np.mean(y)

9.4

In [21]:
np.var(y)

244.50998

In [44]:
%timeit fastfit_negbin(y)

5.44 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
def dispersion_to_rp(mu, phi):
    """ 
    Convert NB2 parameterization to scipy style
    
    Parameters
    ----------
    mu : float 
        NB2 mean.
    phi : float
          NB2 variance. phi=0 corresponds to poisson
    """
    var = mu + phi * mu ** 2
    p = (var - mu) / var
    r = mu ** 2 / (var - mu)
    return {"r": r, "p": p}

def rp_to_dispersion(r, p):
    """ 
    Convert NB2 parameterization to scipy style
    
    Parameters
    ----------
    mu : float 
        NB2 mean.
    phi : float
          NB2 variance. phi=0 corresponds to poisson
    """
    var = mu + phi * mu ** 2
    p = (var - mu) / var
    r = mu ** 2 / (var - mu)
    mu = r*p/(1-p)
    return {"r": r, "p": p}


In [7]:
def fit_nbinom_lbfgs(y, thetamin=1e-1):
    N = len(y)
    ysum = np.sum(y)
    ybar = ysum / N
    mu = ybar

    def nll(theta):
        return (
            -np.sum(gammaln(theta + y))
            + N * gammaln(theta)            
            - ysum * np.log(mu/(theta+mu))            
            - N*theta * np.log(theta/(theta+mu))            
        )

    init_theta = (ybar**2)/(np.var(y)-ybar)
    theta = minimize(nll, init_theta, 
                     bounds=[(thetamin, None)], 
                     method="L-BFGS-B")
    return {"theta": theta.x[0], "mu": mu}

In [74]:
nb = NegativeBinomial(mu=10, theta=200)
y = nb.sample((200,)).numpy()
#fastfit_negbin(y, max_iterations=100)
fit_nbinom_lbfgs(y)

{'theta': 63.805665278180435, 'mu': 10.23}

In [69]:
fit_nbinom_lbfgs(y)

{'theta': 1.779857492789595, 'mu': 9.255}

In [8]:

def fit_nbinom_sm(x):
    params = dm.NegativeBinomial(x, np.ones_like(x)).fit(maxiter=200000, disp=0).params
    mu = np.exp(params[0])
    alpha = params[1]
    r = alpha ** -1
    p = r / (r + mu)
    
    return {"mu": mu, "theta": 1/alpha}


def fit_nbinom_gokceneraslan(X, initial_params=None):
    infinitesimal = np.finfo(np.float).eps

    def log_likelihood(params, *args):
        r, p = params
        X = args[0]
        N = X.size

        # MLE estimate based on the formula on Wikipedia:
        # http://en.wikipedia.org/wiki/Negative_binomial_distribution#Maximum_likelihood_estimation
        result = (
            np.sum(gammaln(X + r))
            - np.sum(np.log(factorial(X)))
            - N * (gammaln(r))
            + N * r * np.log(p)
            + np.sum(X * np.log(1 - (p if p < 1 else 1 - infinitesimal)))
        )

        return -result

    def log_likelihood_deriv(params, *args):
        r, p = params
        X = args[0]
        N = X.size

        pderiv = (N * r) / p - np.sum(X) / (1 - (p if p < 1 else 1 - infinitesimal))
        rderiv = np.sum(psi(X + r)) - N * psi(r) + N * np.log(p)

        return np.array([-rderiv, -pderiv])

    if initial_params is None:
        # reasonable initial values (from fitdistr function in R)
        m = np.mean(X)
        v = np.var(X)
        size = (m ** 2) / (v - m) if v > m else 10

        # convert mu/size parameterization to prob/size
        p0 = size / ((size + m) if size + m != 0 else 1)
        r0 = size
        initial_params = np.array([r0, p0])

    bounds = [(infinitesimal, None), (infinitesimal, 1)]
    optimres = optim(
        log_likelihood,
        x0=initial_params,
        # fprime=log_likelihood_deriv,
        args=(X,),
        approx_grad=1,
        bounds=bounds,
    )

    params = optimres[0]
    mu = (1-params[1])*params[0]/params[1]
    return {"theta": params[0], "prob": params[1], "mu":mu}

In [24]:
nb = NegativeBinomial(mu=10, theta=1/2)
y = nb.sample((200,)).numpy()
fastfit_negbin(y, max_iterations=100)

{'dispersion': 0.4933504381240379, 'mu': 8.835000038146973, 'iterations': 91}

In [9]:
fit_nbinom_sm(y)

{'mu': 9.695, 'theta': 0.4701366616989222}

In [25]:
fit_nbinom_gokceneraslan(y)

/gpfs/commons/home/schoudhary/software/anaconda3/envs/pyfastnbfit/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


{'theta': 0.5254864807830589,
 'prob': 0.05613880002074141,
 'mu': 8.83500003814697}

In [30]:
fit_nbinom_lbfgs(y)

{'theta': 0.493352894261752, 'mu': 8.835000000000006}

In [53]:
params = dispersion_to_rp(mu=100, phi=10)
y = nbinom.rvs(n=params["r"], p=params["p"], size=1000)
fastfit_negbin(y)

/gpfs/commons/home/schoudhary/software/anaconda3/envs/pyfastnbfit/lib/python3.8/site-packages/pyfastnbfit-0.1.0-py3.8-linux-x86_64.egg/pyfastnbfit/pyfastnbfit.py:135: RuntimeWarning: invalid value encountered in double_scalars
  b = xmean / xvar - 1


{'dispersion': nan, 'mu': nan, 'iterations': 0}

In [52]:
fit_nbinom_gokcen(y)

/gpfs/commons/home/schoudhary/software/anaconda3/envs/pyfastnbfit/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


{'size': 1.7040818360556866, 'prob': 0.019855520094885157}

In [35]:
results_fastfit

{'dispersion': nan, 'mu': nan, 'iterations': 2}

In [21]:
mu_diff = defaultdict(list)
theta_diff = defaultdict(list)
time_diff = defaultdict(list)
params = []

mu_seq =  [100, 50, 20, 10, 5, 2, 1, 0.1, 0.01]#, 0.0001]
theta_seq = [100, 50, 20, 10, 5, 2, 1, 0.1, 0.01]#, 0.0001]


fit_params = []
for i in range(1, 5):
    for mu in mu_seq:
        for theta in theta_seq:
            nb = NegativeBinomial(mu=mu, theta=theta)
            y = nb.sample((5000,)).numpy()
            params.append((mu, theta))

            start = time.time()
            results_fastfit = fastfit_negbin(y)
            end = time.time()
            time_diff = end - start
            fit_params.append((mu, theta, "fastnegbin", time_diff,
                               results_fastfit["mu"], results_fastfit["theta"]))


            start = time.time()
            results_gokceneraslan = fit_nbinom_gokceneraslan(y)
            end = time.time()
            time_diff = end - start
            fit_params.append((mu, theta, "fit_nbinom_gokceneraslan",
                               time_diff, results_gokceneraslan["mu"],
                               results_gokceneraslan["theta"]))


            start = time.time()
            results_sm = fit_nbinom_sm(y)
            end = time.time()
            time_diff = end - start
            fit_params.append((mu, theta, "fit_nbinom_sm",
                       time_diff, results_sm["mu"],
                       results_sm["theta"]))

            start = time.time()
            results_lbfgs = fit_nbinom_lbfgs(y)
            end = time.time()
            time_diff = end - start
            fit_params.append((mu, theta, "fit_nbinom_lbfgs",
               time_diff, results_lbfgs["mu"],
               results_lbfgs["theta"]))
            
            start = time.time()
            results_fp = fit_nb_fixedpoint(y)
            end = time.time()
            time_diff = end - start
            fit_params.append((mu, theta, "fit_nb_fixedpoint",
               time_diff, results_fp["mu"],
               results_fp["theta"]))
            
            
            


RuntimeError: Failed to converge after 200 iterations, value is nan

In [22]:
mu

100

In [23]:
theta

0.1

In [ ]:
df = pd.DataFrame(fit_params)
df.columns = ["mu", "theta", "method", "time", "mu_pred", "theta_pred"]
df["mu_diff"] = df["mu_pred"] - df["mu"]
df["theta_diff"] = df["theta_pred"] - df["theta"]
df["log_time"] = np.log10(df["time"])
df.loc[df.theta_pred>10**4, "theta_pred"] = 10**3
df["theta_diff"] = df["theta_pred"] - df["theta"]

import seaborn as sns
sns.violinplot(x="method", y="log_time", data=df)

In [ ]:
sns.violinplot(x="method", y="mu_diff", data=df)

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(x="method", y="theta_diff", data=df, ax=ax)
#ax.set(yscale="log")


In [ ]:
df.loc[df.theta_diff>100]

In [96]:
import plotnine

In [ ]:
p1 <- ggplot(predictions.factored, aes(N, estimate)) +  
geom_line(aes(color = true, linetype=true), size=1.5) +  
scale_color_brewer(palette="Paired") + 
geom_hline(yintercept = theta_seq, linetype="dashed") + 
scale_x_log10() +
scale_y_log10() +
labs(x = expression(N), 
   y = expression(theta["estimated"]))
p1

In [ ]:
sns.(x="method", y="theta_diff", data=df, ax=ax)


In [3]:
a = 30
b = 7
p = 1 / (b + 1)
y = nbinom.rvs(n=a, p=1 / (b + 1), size=100)
fastfit_negbin(y)

{'dispersion': 33.30970983755638, 'mu': 208.07999999999996, 'iterations': 11}

In [6]:
theta = 10
p = 0.5
mu = theta*p/(1-p)
print(mu, theta)
y = np.random.negative_binomial(n = 10, p = 0.1, size = 100)

fastfit_negbin(y)

10.0 10


{'dispersion': 10.886168685322538, 'mu': 91.18, 'iterations': 9}

In [3]:
print(",".join(map(lambda x: str(x), y)))

205,185,185,245,217,336,281,273,176,180,226,285,133,221,206,252,238,152,189,196,150,193,215,166,181,163,183,207,217,204,156,206,148,183,163,150,278,206,236,185,226,170,248,219,193,197,300,219,190,214,177,194,192,194,221,197,235,255,156,187,251,287,196,214,213,269,181,180,193,249,226,223,222,212,207,196,241,201,181,173,266,147,314,230,220,148,208,182,154,177,187,188,269,180,178,248,206,216,192,244


In [4]:
# test a poisson
#x4 <- rnegbin(500, mu = 10, theta = 400)
y = [6,10,8,11,8,12,9,12,13,10,10,9,7,6,10,5,8,10,9,9,10,12,3,9,6,9,8,11,6,6,11,9,12,14,12,11,6,7,13,8,7,7,13,6,5,17,7,11,7,12,6,16,8,6,10,8,8,8,11,6,8,9,10,8,11,17,10,10,10,13,13,9,10,6,5,9,5,9,13,11,8,8,8,7,10,9,14,6,11,11,8,4,11,7,5,6,13,13,8,9,10,8,12,12,4,9,11,16,11,8,9,6,10,6,16,18,7,10,12,9,10,8,4,6,6,14,9,11,11,9,10,9,12,11,7,9,7,10,8,10,12,8,14,7,14,9,3,7,10,10,11,8,5,10,12,8,10,11,13,14,9,7,13,14,8,10,18,8,14,12,10,4,19,7,5,7,12,6,8,9,8,11,13,10,12,12,8,7,11,10,9,6,11,6,6,10,10,5,8,14,9,12,11,10,8,10,8,9,10,14,12,13,5,7,16,13,11,13,14,8,9,9,17,11,12,9,10,7,9,12,16,9,15,12,5,9,9,5,16,6,8,10,10,10,9,13,16,12,11,15,16,8,7,13,13,16,8,7,9,11,10,10,9,7,7,8,13,7,7,8,10,16,5,10,11,8,7,12,18,8,13,11,12,8,11,9,8,12,7,11,8,7,12,16,6,7,12,10,9,17,10,15,11,10,8,9,12,11,10,13,10,10,11,15,11,6,10,8,10,10,6,9,17,7,13,10,14,7,12,12,6,13,9,12,5,7,10,12,8,5,11,11,12,14,11,15,6,11,14,16,6,11,10,13,7,7,2,12,11,13,9,9,11,16,8,14,6,14,7,4,10,6,11,11,9,14,11,13,10,3,8,16,17,13,8,9,7,9,15,17,8,3,5,9,12,11,8,15,10,19,13,12,12,9,14,12,12,10,10,9,12,13,8,6,9,10,10,8,8,6,12,7,11,7,6,9,12,7,9,13,7,7,11,9,6,9,10,10,7,14,11,8,14,7,9,12,6,6,15,10,13,12,2,4,9,13,11,5,11,4,5,12,12,6,8,9,9,10,6,9,9,10,6,9,6,4,9,12,12,8,7,9,12,9,6,10,9,15,14,14,9,16,12,9,11,6,10,11,7,3]
fastfit_negbin(y)

{'a': 405.471773981185,
 'b': 0.024134848904313857,
 'p': 0.9764339149965116,
 'dispersion': 17205.733320624266,
 'mu': 9.785999999999998,
 'iterations': 20}